[1] 모듈 로딩

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torchinfo import summary
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [3]:
## 모델 파일 관련
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/lang/MCF/'
# 저장 파일명
SAVE_FILE=SAVE_PATH+'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL=SAVE_PATH+'model_all.pth'

[2] 모델 로딩 - 모델 전체 파일 사용

In [4]:
# 은닉층을 리스트로 넘겨줌
class MCFmodel(nn.Module):
    def __init__(self, in_in, out_out, h_list=[]):
        super().__init__()

        self.input_layer = nn.Linear(in_in,h_list[0])      #입력은 피쳐수
        self.h_layers = nn.ModuleList()

        for i in range(len(h_list)-1):
            self.h_layers.append(nn.Linear(h_list[i],h_list[i+1]))
        
        self.out_layers = nn.Linear(h_list[-1], out_out)  # 타겟수
    
    def forward(self, x):
        y = F.relu(self.input_layer(x))  

        for h_layer in self.h_layers:
            y=F.relu(h_layer(y))   
        
        return self.out_layers(y) # 다중분류

In [5]:
# 커스텀모델은 모델을 임포트해야 가능하다
if os.path.exists(SAVE_MODEL): # 해당경로에 있다면
    model= torch.load(SAVE_MODEL, weights_only=False)
else:
    print(f'{SAVE_MODEL} 파일이 존재하지 않습니다. 다시 확인하세요. ') 

In [6]:
summary(model)

Layer (type:depth-idx)                   Param #
MCFmodel                                 --
├─Linear: 1-1                            270
├─ModuleList: 1-2                        --
│    └─Linear: 2-1                       330
│    └─Linear: 2-2                       620
├─Linear: 1-3                            84
Total params: 1,304
Trainable params: 1,304
Non-trainable params: 0

[3] 예측

여기서는 한개의 값만 예측

In [7]:
TEST_CSV = 'lang_test.csv'

In [8]:
testDF = pd.read_csv(TEST_CSV, header=None)

In [20]:
test = testDF.iloc[[3]]

In [47]:
# class langDataset(Dataset):
#     def __init__(self, featureDF, targetDF):
#         self.featureDF = featureDF
#         self.targetDF = targetDF
#         self.n_rows = featureDF.shape[0]
#         self.n_features=featureDF.shape[1]
    
#     def __len__(self):
#         return self.n_rows

#     def __getitem__(self, index):
#         # 넘파이를 텐서로
#         featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
#         targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)        
#         # 피쳐와 타겟 반환
#         return featureTS, targetTS      

In [21]:
# 피쳐 타겟 추출
test_featureDF, test_targetDF = test[test.columns[1:]], test[test.columns[:1]]

In [22]:
dataTS = torch.FloatTensor(test_featureDF.values)

In [23]:
dataTS

tensor([[0.0719, 0.0114, 0.0385, 0.0403, 0.1394, 0.0122, 0.0154, 0.0184, 0.0795,
         0.0042, 0.0024, 0.0587, 0.0332, 0.0708, 0.0699, 0.0496, 0.0069, 0.0641,
         0.0730, 0.0663, 0.0487, 0.0136, 0.0029, 0.0043, 0.0034, 0.0012]])

In [24]:
from function_list import predict_data

In [25]:
model.eval()
with torch.no_grad():

    # 추론/평가
    pre_val=model(dataTS)


In [26]:
idx = torch.argmax(pre_val, dim=1).item()

In [27]:
class_name = ['en', 'fr', 'id', 'tl']

In [28]:
predict = class_name[idx]

In [29]:
actual = test_targetDF.iloc[0][0]

In [30]:
print(f'actual = > {actual} , predict => {predict} ')

actual = > fr , predict => fr 
